# **Fiber Photometry and Optogenetic Stimulation in SN GABAergic Neurons**

This tutorial shows how to access and process data for the study detailed in the [preprint]() MISSING PREPRINT

This dataset contains sessions with fiber photometry recordings, optogenetic stimulation TTL signals, shock delivery TTL signals, behavioral videos, and histological verification images, along with subject and session metadata.

Contents:

- [Streaming Data](#stream-nwb)
- [Reading Data](#read-nwb)
- [Access Fiber Photometry Signal](#access-photometry)
- [Access Optogenetic and Shock TTLs](#access-ttls)
- [Access Behavioral Videos](#access-behavior) MISSING 
- [Access Histology Images](#access-histology) MISSING
- [Access Demodulated Fiber Photometry signals](#access-demod)

A schematic representation of ... : MISSING  

![Alt text](./hnasko-dataset-structure.png)  

---

# Select the subject and session to load

In [ ]:
dandi_id = ""
subject_id = "C4561"
session_id = "240117-163347"

# Streaming an NWB file <a id="stream-nwb"></a>

This section demonstrates how to access the files on the [DANDI Archive](https://dandiarchive.org) without downloading them. Based on the [Streaming NWB files](https://pynwb.readthedocs.io/en/stable/tutorials/advanced_io/streaming.html) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

An [NWBFile](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment. Each NWBFile must have a `session description`, `identifier`, and `session start time`.

The `dandi.dandiapi.DandiAPIClient` can be used to get the S3 URL of the NWB file stored in the DANDI Archive.

```python
from dandi.dandiapi import DandiAPIClient

nwbfile_path = f"sub-{subject_id}/sub-{subject_id}_ses-{session_id}.nwb"

dandiset_id = ""
with DandiAPIClient() as client:
    #This line is necessary when the dataset is in embargoed mode and only owners can view the data, once it will be published this line can be removed.
    client.dandi_authenticate() 
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(nwbfile_path)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)

import h5py
import remfile
from pynwb import NWBHDF5IO

file = remfile.File(s3_url)
h5_file = h5py.File(file, "r")
io = NWBHDF5IO(file=h5_file, load_namespaces=True)
```

# Reading an NWB file locally<a id="read-nwb"></a>


This section demonstrates how to read an NWB file using `pynwb`.

Based on the [NWB File Basics](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#sphx-glr-tutorials-general-plot-file-py) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

An [NWBFile](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment. Each NWBFile must have a `session description`, `identifier`, and `session start time`.

Reading is carried out using the [NWBHDF5IO](https://pynwb.readthedocs.io/en/stable/pynwb.html#pynwb.NWBHDF5IO) class. To read the NWB file use the read mode ("r") to retrieve an NWBFile object.


In [ ]:
from pynwb import NWBHDF5IO

#adapt the path
nwbfile_path = f"/Users/daphnedequatrebarbes/Documents/Catalystneuro/hnasko-lab-to-nwb/nwb_output/sub-{subject_id}_ses-{session_id}.nwb"
io = NWBHDF5IO(path=nwbfile_path, load_namespaces=True)

Importantly, the `session start time` is the reference time for all timestamps in the file. For instance, an event with a timestamp of 0 in the file means the event occurred exactly at the session start time.

The `session_start_time` is extracted from all_sessions.Session datajoint table.

In [ ]:
nwbfile = io.read()
nwbfile

In [ ]:
nwbfile.session_start_time

In [ ]:
nwbfile.experiment_description

In [ ]:
nwbfile.session_description

# Access subject <a name="access-subject"></a>

This section demonstrates how to access the [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field in an NWB file.

The [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field can be accessed as `nwbfile.subject`.


In [ ]:
nwbfile.subject

# Access Fiber Photometry Data <a id="access-photometry"></a>

This section demonstraces how to access the raw Fiber Photometry data.

`NWB` organizes data into different groups depending on the type of data. Groups can be thought of as folders within the file. Here are some of the groups within an NWBFile and the types of data they are intended to store:

- `acquisition`: raw, acquired data that should never change
- `processing`: processed data, typically the results of preprocessing algorithms and could change

## Raw Modulated Fiber Photometry signal 

The raw fiber photometry response data is stored in [pynwb.ophys.FiberPhotometryResponseSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.FiberPhotometryResponseSeries) object, which are added to `nwbfile.acquisition`.

The data in FiberPhotometryResponseSeries is stored as a two-dimensional array:

The first dimension corresponds to time (individual samples).
The second dimension corresponds to recorded channels.
Each row in the array represents a single time point, and each column corresponds to the signal recorded from a specific channel.



In [ ]:
fiber_photometry_response_series = nwbfile.acquisition["raw_modulated_signal"]
fiber_photometry_response_series

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Define your window (start and end indices)
start_idx = 1000
end_idx = 1500

# Get the data and timestamps properly
data = np.array(fiber_photometry_response_series.data[start_idx:end_idx])
timestamps = fiber_photometry_response_series.get_timestamps()[start_idx:end_idx]

# Create a single plot
fig, ax = plt.subplots(figsize=(8, 3), dpi=300)

# Plot the raw fluorescence
ax.plot(timestamps, data, color="green", linewidth=0.8, label='Raw Modulated Fiber Photometry Signal')

# Style the plot
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.legend(frameon=False, bbox_to_anchor=(.95, 1), loc='upper left', prop={'size': 8})
ax.tick_params(axis='both', labelsize=8)

plt.xlabel('Time (s)', fontsize=8)
plt.ylabel('Fluorescence (a.u.)', fontsize=8)

plt.show()


## Raw Modulated Fiber Photometry metadata 
The fiber photometry metadata includes the type of indicator(s), optical fiber(s), excitation source(s), photodector(s), dichroic mirror(s), and optical filter(s) that were used to construct a single fluorescence signal.

The metadata is stored in a `FiberPhotometryTable` object using [`ndx-fiber-photometry`](https://github.com/catalystneuro/ndx-fiber-photometry) and is added to `nwbfile.lab_meta_data`. It can be accessed as `nwbfile.lab_meta_data["FiberPhotometry"].fiber_photometry_table`.

In [ ]:
fiber_photometry_table_region = nwbfile.lab_meta_data["fiber_photometry"].fiber_photometry_table[:]
fiber_photometry_table_region

The metadata on the optical fiber used to record the GCaMP fluorescence is added to `nwbfile.devices` and can be accessed as `nwbfile.devices["Fiber1"]` or can be accessed from the referenced optical fiber in the `fiber_photometry_table_region` of the `FiberPhotometryResponseSeries`.

In [ ]:
fiber_photometry_table_region["optical_fiber"][0]

In [ ]:
fiber_photometry_table_region["indicator"][0]

In [ ]:
fiber_photometry_table_region["excitation_source"][0]

In [ ]:
fiber_photometry_table_region["photodetector"][0]

In [ ]:
fiber_photometry_table_region["dichroic_mirror"][0]

In [ ]:
fiber_photometry_table_region["excitation_source"][0]

# Access Optogenetic Stimulation Data <a name="access-ttls"></a>

This section demonstrates how to access the optogenetic stimulation events recorded during the experiment.

In NWB format, externally generated stimuli such as laser pulses are typically stored as TimeSeries objects within the nwbfile.stimulus group. These signals capture event timing and are often recorded as TTL pulses synchronized with the acquisition system.

In this dataset, optogenetic stimulation was applied using TTL signals with varying frequencies (5 Hz, 10 Hz, 20 Hz, 40 Hz) and durations (250 ms, 1 s, 4 s), across multiple trials. The TTL events are used to align stimulation timing with fluorescence recordings for downstream analysis.

The optogenetic TTL timestamps can be accessed as:

In [ ]:
optogenetic_stimulus_interval = nwbfile.stimulus["optogenetic_stimulus_interval"]
optogenetic_stimulus_interval

In [ ]:

optogenetic_stimulus_interval.to_dataframe().head(5)

In [ ]:
optogenetic_series = nwbfile.stimulus["optogenetic_series"]
optogenetic_series.data

In [ ]:
optogenetic_series

In [ ]:
data = optogenetic_series.data[:]
timestamps = optogenetic_series.timestamps[:]

print(f"Data shape: {data.shape}")
print(f"Timestamps shape: {timestamps.shape}")
print(f"Duration: {timestamps[-1] - timestamps[0]:.2f} seconds")

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(timestamps, data, lw=0.8)
plt.xlabel("Time (s)")
plt.ylabel("Laser Power (W)")
plt.title("Optogenetic Stimulation Time Series")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Define manual time windows 
block_windows = [
    (100, 600),   # Block 1
    (700, 1200),  # Block 2
    (1300, 1750)  # Block 3
]

# Plot each block
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharey=True)

for i, (start, end) in enumerate(block_windows):
    mask = (timestamps >= start) & (timestamps <= end)
    axes[i].plot(timestamps[mask], data[mask])
    axes[i].set_title(f"Optogenetic Stimulation - Block {i+1}")
    axes[i].set_xlabel("Time (s)")
    axes[i].set_ylabel("Laser Power (W)")
    axes[i].grid(True)

plt.tight_layout()
plt.show()

# Access Behavioral Videos <a name="access-behavior"></a>


# Access Histology Images <a name="access-histology"></a>

# Accessing the Demodulated Fiber Photometry Response Data <a name="access-demod"></a>

The demodulated fluorescence signals from fiber photometry recordings are stored in `nwbfile.processing["ophys"]`.

## Demodulated Fiber Photometry Signals

In NWB, the [`FiberPhotometryResponseSeries`](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.FiberPhotometryResponseSeries) class is used to store time-varying fluorescence signals from one or more optical channels, such as calcium-dependent (e.g., 465 nm) and isosbestic control (e.g., 405 nm) signals.

We can access the demodulated fiber photometry data for each channel using:

In [ ]:
nwbfile.processing["ophys"]["calcium_signal"]
nwbfile.processing["ophys"]["isosbestic_signal"]

In [ ]:
# Access both signals
module = nwbfile.processing["ophys"]
isosbestic = module["isosbestic_signal"]
calcium = module["calcium_signal"] 

# Extract data and sampling info
rate = isosbestic.rate
start_time = isosbestic.starting_time

# Load data
isosb_data = isosbestic.data[:]
calcium_data = calcium.data[:]

# Build timestamps (assume both have the same sampling rate and start time)
timestamps = start_time + np.arange(isosbestic.data.shape[0]) / rate

# Select a 10-second window
duration = 10  # seconds
samples = int(duration * rate)

# Plot both signals
plt.figure(figsize=(14, 5))
plt.plot(timestamps[:samples], isosb_data[:samples], label="Isosbestic (405 nm)", alpha=0.7)
plt.plot(timestamps[:samples], calcium_data[:samples], label="Calcium (465 nm)", alpha=0.7)
plt.xlabel("Time (s)")
plt.ylabel("Fluorescence (a.u.)")
plt.title("Demodulated Fiber Photometry Signals (first 10 seconds)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Demodulated Fiber Photometry metadas

In [ ]:
# Access the isosbestic signal object
isosbestic = nwbfile.processing["ophys"]["isosbestic_signal"]
table = isosbestic.fiber_photometry_table_region

table.to_dataframe()